![title](../images/logo_nao_digital.png)

# Tema 3: Modelos básicos de Aprendizaje de Máquina y Validación cruzada 

Familiarizarse con otros modelos de aprendizaje de máquina y con los conceptos de validación cruzada para seleccionar el modelo predictivo de la demanda de renta de bicicletas.

## Datos de BikerPro

Cómo se ha mencionado en el Anexo A, se ha provisto la siguiente información al equipo del `Ministerio de Análisis de Datos`:

| Nombre 	| Descripción 	| Tipo 	|
|---	|---	|---	|
| Date 	| Fecha (con año, mes y dia) 	| Fecha 	|
| Rented Bike Count 	| Cantidad de bicicletas rentadas por fecha y hora  	| Entero 	|
| Hora 	| Hora del día 	| Entero 	|
| Temperature 	| Temperatura promedio en grados centígrados 	| Real 	|
| Humidity  	| Nivel de humedad en el ambiente, en porcentaje. 	| Real 	|
| Wind speed 	| Velocidad del viento, en metros sobre segundo 	| Real 	|
| Visibility 	| Medida del nivel de visibilidad de 10 metros (mayor visibilidad   implica mejores condiciones meteorológicas para ver de lejos a un objeto). 	| Real 	|
| Dew point temperature 	| Temperatura de punto de rocío, es decir temperatura más alta en que   el agua se condensa, medida en grados centígrados 	| Número Real 	|
| Solar radiation  	| Es una medida de la radiación solar promedio existente en el   ambiente, medida Megajoules / metros cuadrados 	| Número Real 	|
| Rainfall 	| Se refiere a la cantidad de precipitación pluvial que hay a nivel   de suelo, medida en milímetros 	| Número Real 	|
| Snowfall 	| Nivel de caída de nivel (en centímetros) 	| Numérico 	|
| Seasons 	| Estación del año en inglés (Winter, Spring, Summer, Autumn) 	| Texto 	|
| Holiday 	| Indica si esa fecha y hora es festiva 	| Texto 	|
| Functional Day 	| Describe si en esa hora específica el servicio de renta se   encontraba en funcionamiento (se podía rentar un equipo) o no 	| Texto    	|




Este documento se desarrollarán scripts en Python que permitan entrenar familias de diferentes modelo de aprendizaje supervisado y predecir con el mejor de ellos a la demanda de renta de bicicletas de BikePro a partir de principios de Aprendizaje de Máquina.

## 2. Librerias de trabajo

In [183]:
# Instala libreria Pandas si no la tenemos
#pip install pandas seaborn scikit-learn -y

In [184]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# importa clase de Python para instanciar el modelo KNN
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline

import warnings
warnings.filterwarnings('ignore')

## 3. Lectura de datos

Primero nos encargaremos de leer los datos, indicando a Python donde se encuentra la carpeta que contiene los datos y los nombres de los archivos relevantes para el análisis.

In [185]:
#  Indicamos la ruta a la carpeta de de tu computadora 
# donde se ubican los datos de BikerPro
# Ejemplo: "C:\Usuarios\[tu nombre]\Descargas"

DATA_PATH=""

Ahora procederemos a definir una variable que indique el nombre del archivo junto con su extensión (por ejemplo, `.csv`):

In [186]:
FILE_BIKERPRO = 'SeoulBikeData.csv'

Echaremos mano de la utilidad `os.path.join` de Python que indicar rutas en tu computadora donde se ubican archivos, así Pandas encontrá los archivos de datos.


**Ejemplo**

A continuación mostraremos un ejemplo leyendo el archivo `SeoulBikeData.csv`:

In [187]:
# Ejemplo
print(f"Ruta del archivo: {FILE_BIKERPRO}")
print(os.path.join(DATA_PATH, FILE_BIKERPRO))

Ruta del archivo: SeoulBikeData.csv
/Users/cesar/sandbox/tecmilenio/DN_PRO_20/data/SeoulBikeData.csv


In [188]:
# Leemos con pandas
bikerpro = pd.read_csv(
    os.path.join(DATA_PATH, FILE_BIKERPRO),
    encoding = "ISO-8859-1"
    )

In [189]:
bikerpro.head(10)

,Date,Rented Bike Count,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(°C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day
0,01/12/2017,254,0,-5.2,37,2.2,2000,-17.6,0.00,0.0,0.0,Winter,No Holiday,Yes
1,01/12/2017,204,1,-5.5,38,0.8,2000,-17.6,0.00,0.0,0.0,Winter,No Holiday,Yes
2,01/12/2017,173,2,-6.0,39,1.0,2000,-17.7,0.00,0.0,0.0,Winter,No Holiday,Yes
3,01/12/2017,107,3,-6.2,40,0.9,2000,-17.6,0.00,0.0,0.0,Winter,No Holiday,Yes
4,01/12/2017,78,4,-6.0,36,2.3,2000,-18.6,0.00,0.0,0.0,Winter,No Holiday,Yes
5,01/12/2017,100,5,-6.4,37,1.5,2000,-18.7,0.00,0.0,0.0,Winter,No Holiday,Yes
6,01/12/2017,181,6,-6.6,35,1.3,2000,-19.5,0.00,0.0,0.0,Winter,No Holiday,Yes
7,01/12/2017,460,7,-7.4,38,0.9,2000,-19.3,0.00,0.0,0.0,Winter,No Holiday,Yes
8,01/12/2017,930,8,-7.6,37,1.1,2000,-19.8,0.01,0.0,0.0,Winter,No Holiday,Yes
9,01/12/2017,490,9,-6.5,27,0.5,1928,-22.4,0.23,0.0,0.0,Winter,No Holiday,Yes


Por simplicidad, limpiaremos los encabezado eliminando unidades, quitando espacios y transformando las letras a minusculas:

In [190]:
# formato de columnas en crudo
raw_columns = list(bikerpro.columns)
raw_columns

['Date',
 'Rented Bike Count',
 'Hour',
 'Temperature(°C)',
 'Humidity(%)',
 'Wind speed (m/s)',
 'Visibility (10m)',
 'Dew point temperature(°C)',
 'Solar Radiation (MJ/m2)',
 'Rainfall(mm)',
 'Snowfall (cm)',
 'Seasons',
 'Holiday',
 'Functioning Day']

In [191]:
clean_columns = [
    x.lower().\
        replace("(°c)", '').\
        replace("(%)", '').\
        replace(" (m/s)", '').\
        replace(" (10m)", '').\
        replace(" (mj/m2)", '').\
        replace("(mm)", '').\
        replace(" (cm)", '').\
        replace(" ", '_')
    for x in bikerpro.columns
    ]

In [192]:
# Ahora asignamos los nuevmos nombres de columnas para el análisis
bikerpro.columns = clean_columns

Además convertiremos al formato adecuado a la columna de fecha:

In [193]:
bikerpro['date'] = pd.to_datetime(bikerpro['date'], format='%d/%m/%Y')

Como sabemos el proceso anterior nos convierte limpia las columnas para dejarlas en el formato descrito a continuación:

In [194]:
bikerpro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   date                   8760 non-null   datetime64[ns]
 1   rented_bike_count      8760 non-null   int64         
 2   hour                   8760 non-null   int64         
 3   temperature            8760 non-null   float64       
 4   humidity               8760 non-null   int64         
 5   wind_speed             8760 non-null   float64       
 6   visibility             8760 non-null   int64         
 7   dew_point_temperature  8760 non-null   float64       
 8   solar_radiation        8760 non-null   float64       
 9   rainfall               8760 non-null   float64       
 10  snowfall               8760 non-null   float64       
 11  seasons                8760 non-null   object        
 12  holiday                8760 non-null   object        
 13  fun

**Creación de variables**

Podemos a su vez definir variables auxiliares relacionadas al tiempo:

In [195]:
# Define el dia de la semana como variable categorica
bikerpro['weekday'] = bikerpro['date'].dt.weekday
bikerpro['weekday'] = bikerpro['weekday'].astype('category')

# Define el dia de la semana como variable categorica
bikerpro['month'] = bikerpro['date'].dt.month
bikerpro['month'] = bikerpro['month'].astype('category')

In [196]:
# Variable indicadora de si el dia 
bikerpro['is_weekend'] = np.where(bikerpro['date'].dt.weekday> 4,1,0)

**División del conjunto de datos en entrenamiento y prueba**

In [197]:
# datos ordenados
X = bikerpro.sort_values(['date', 'hour'])

In [198]:
# Columnas del clima
weather_cols = [
    'temperature', 
    'humidity',
    'humidity',
    'wind_speed',
    'visibility',
    'dew_point_temperature',
    'solar_radiation',
    'rainfall',
    'snowfall'
    ]

# columna objectivo a predecir
target_col = ['rented_bike_count']

**Dividimos al conjunto en entrenamiento y prueba**

In [199]:
# Datos de entrenamiento
X_train = X.loc[: X.shape[0]-1440,:].drop(target_col, axis=1)
y_train = X.loc[: X.shape[0]-1440,:][target_col]

# Datos de entrenamiento
X_test = X.loc[X.shape[0]-1440+1:,:].drop(target_col, axis=1)
y_test = X.loc[X.shape[0]-1440+1:,:][target_col]

In [200]:
X_test

,date,hour,temperature,humidity,wind_speed,visibility,dew_point_temperature,solar_radiation,rainfall,snowfall,seasons,holiday,functioning_day,weekday,month,is_weekend
7321,2018-10-02,1,12.5,74,1.9,1992,7.9,0.0,0.0,0.0,Autumn,No Holiday,No,1,10,0
7322,2018-10-02,2,12.3,75,1.6,1840,7.9,0.0,0.0,0.0,Autumn,No Holiday,No,1,10,0
7323,2018-10-02,3,11.8,78,0.3,1843,8.0,0.0,0.0,0.0,Autumn,No Holiday,No,1,10,0
7324,2018-10-02,4,11.2,80,0.3,1236,7.8,0.0,0.0,0.0,Autumn,No Holiday,No,1,10,0
7325,2018-10-02,5,10.8,82,0.3,1778,7.8,0.0,0.0,0.0,Autumn,No Holiday,No,1,10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2018-11-30,19,4.2,34,2.6,1894,-10.3,0.0,0.0,0.0,Autumn,No Holiday,Yes,4,11,0
8756,2018-11-30,20,3.4,37,2.3,2000,-9.9,0.0,0.0,0.0,Autumn,No Holiday,Yes,4,11,0
8757,2018-11-30,21,2.6,39,0.3,1968,-9.9,0.0,0.0,0.0,Autumn,No Holiday,Yes,4,11,0
8758,2018-11-30,22,2.1,41,1.0,1859,-9.8,0.0,0.0,0.0,Autumn,No Holiday,Yes,4,11,0


## 4. Modelos Básicos de Aprendizaje de Máquina

Esta sección aborda algunos de los principales modelos usados en problemas de aprendizaje supervisado.

### 4.1 Modelos de Regresión Lineal

Los modelos de regresión son modelos estadísticos que asumen la existencia de una relación lineal entre las características del problema y la variable a predecir, es decir, la variable objetivo se puede describir de la forma

$$y_i = \beta_0 + \beta_1 x_{i1}  + \beta_2 x_{i2} + \ldots + \beta_n x_{in}+ \epsilon_i$$

Donde los coeficientes $\beta_i$ son parámetros por determinarse y $\epsilon_i$ es una representación del ruido en la relación entre $y_i$ y $x_{i1}, \ldots, x_{in}$. En la jerga estadística los variables reciben el nombre de regresores.

Debe notarse que dado que los parámetros deben estimarse existen supuestos de tipo estadístico para que la relación lineal entre las variables y el objetivo se aproxime de buena manera, es decir, el residuno, nombre que recib el error entre $y$ y la $x_{i1}, \ldots, x_{1n}$, que generalmente se describe en termino del error cuadrático medio.

La teoría estadística señala algunos de los supuestos que deben cumplirse para que la regresión sea un modelo con buen funcionamiento (es decir controlando el sesgo y la varianza de la predicción):

* **Linealidad:** La relación entre la variable dependiente $Y$ y las variables independientes $X$ debe ser lineal.
* **Normalidad de los residuos:** Los residuos deben seguir una distribución normal, este supuesto se puede relajar a una distribución aproximadamente normal.
* **Homogeneidad de la varianza de los residuos:** Los residuos deben tener una varianza constante (homocedasticidad), es decir se espera que la variación de los residuos no se dispare en alguna región del espacio.
* **Independencia de los residuos:** Los residuos deben ser independientes los unos de los otros, es decir  no deben estar correlacionados entre sí.

Por otro lado la estimación de los coeficientes $\beta_i$ se realiza con técnicas de análisis numérico y que siguen principios estadísticos que, cumpliendose los supuestos anteriores, aseguran que el modelo obtenido tenga la combinación de coeficientes que asegura una varianza mínimo (Teorema de Gauss-Markov). Normalmente los paquetes de cómputo científico realizan los cálculos correspondientes, pero la verificación de dichos supuestos es responsabilidad de quien realiza el análisis.

Es dable mencionar que los modelos de regresión son herramientas poderosas de aprendizaje supervisado, pero tienen algunos puntos a considerar:

    * Se ven afectados fuertemente por la presencia de valores atípicos y la presencia de escala distintas en los datos, por lo que es común pre-procesar los datos para eliminar valores ruidosos y asegurando que todos sus componentes tengan órdenes de magnitud comparables.
    * Requieren que los variables dentro de los mismos sean linealmente independientes en el sentido del álgebra lineal, normalmente un buen análisis de correlación puede ayudar a desechar aquellas variables entre las que exista correlación lineal para mejorar su desempeño
    * Son muy flexibles, típicamente puede transformarse para capturar relaciones no lineales.
    * Generalmente la cantidad de variables que se puede incluir está limitada por la cantidad de puntos en el conjunto de entrenamiento.
    * Si todas las variables tienen la misma escala, los coeficientes se pueden interpretar como el efecto que tiene una característica sobre la variable objetivo, considerando que todas las demás características se quedan fija (*ceteris paribus*).

#### 4.1.1 Modelos de Regresión Lineal y Regularización

Para mejorar evitar el sobreajuste de estos modelos, existen técnicas que penalizan la complejidad del modelo, es decir el valor que pueden tomar los coeficientes y que en general inducen mejores resultados para predecir.

Los modelos de regresión que incluyen penalizaciones sobre los coeficientes tienen denominaciones especiales:

* **Lasso:** Este modelo es similar al descrito arriba pero considerando que los coefientes se encuentre en una región definida por $\sum |\beta_i| \leq K_1$ donde $K_1$ es alguna constante. Esta métrica generalmente hace que algunos de los coeficientes $\beta_i$ sean cercanos a cero o bien se anulen.
* **Ridge:** Similar al modelo previo, pero considerando que los coefientes se encuentre en una región definida por $\sum |\beta_i|^2 \leq K_2$ donde $K_2$ es alguna constante.
* **Elastic Net:** En este caso, se pide que los coeficientes satisfagan la restricción $\sum |\beta_i| + \sum |\beta_i|^2 \leq K_3$ donde $K_3$ es cierta constante.


Una forma alternativa de describir lo anterior, es la formulación de Lagrange los problemas de minimización con restriciones, donde esencialmente se busca minimizar la expresión para encontrar el error más el valor de los parámetros

* **Lasso:** $$||y - \beta X||^2_2 + \alpha ||\beta||^2_2$$ donde el coeficiente $\alpha$ es un parámetros que controla la restricción del tamaño de $\beta$. 
* **Ridge:** $$||y - \beta X||^2_2 + \alpha ||\beta||^2_1$$ donde el coeficiente $\alpha$ es un parámetros que controla la restricción del tamaño de $\beta$. 
* **Elastic NEt:** $$||y - \beta X||^2_2 + \alpha_1 ||\beta||^2_1 + \alpha_2  ||\beta||^2_2$$ donde el coeficiente $\alpha$ es un parámetros que controla la restricción del tamaño de $\beta$. (La parametrización de Sklear es equivalente, ver https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html#sklearn.linear_model.ElasticNet)

En Sklearn, dichos modelos se encuentran disponibles en las clases `LinearRegression`, `Lasso`, `Ridge` y `ElasticNet`. En Python, los valores de $\alpha_i$ y sus reparamtrizaciónes se utilizan para controlar que tantos se restringe a los coeficientes para controlar el sobreajuste. Al ser parámetros que no dependen específicamente de los datos, de denominar **hiper-parámetros**.

In [201]:
from sklearn.linear_model import (
    LinearRegression,
    Lasso,
    Ridge,
    ElasticNet
)

# Instanciamos los diferentes modelos
lr = LinearRegression()
lr_lasso = Lasso(alpha=0.5)
lr_ridge = Ridge(alpha=0.5)
lr_elastict = ElasticNet(alpha=0.5, l1_ratio=0.3)

models_regresion = {
    'regression': lr,
    'lasso': lr_lasso,
    'ridge': lr_ridge,
    'elastic': lr_elastict
    }

In [202]:
for model_name in models_regresion.keys():

    print("Modelo:", model_name)
    model = models_regresion[model_name]

    # Ajusta el modelo con los datos de prueba
    model.fit(X_train[weather_cols],y_train)

    y_train_pred = model.predict(X_train[weather_cols])
    y_test_pred = model.predict(X_test[weather_cols])

    # error en conjunto de entrenamiento y prueba
    error_train = mean_squared_error(y_train, y_train_pred)
    error_test = mean_squared_error(y_test, y_test_pred)

    # errores
    print("Error RSME en train:", round(error_train,2) )
    print("Error RSME en test:", round(error_test,2) )

    print("----------------------------------------------")

    


Modelo: regression
Error RSME en train: 242649.15
Error RSME en test: 297561.94
----------------------------------------------
Modelo: lasso
Error RSME en train: 242653.05
Error RSME en test: 297395.13
----------------------------------------------
Modelo: ridge
Error RSME en train: 242649.15
Error RSME en test: 297560.67
----------------------------------------------
Modelo: elastic
Error RSME en train: 244621.81
Error RSME en test: 296198.37
----------------------------------------------


**Preguntas**

* En los resultados se aprecia que algunos modelos con regulzación tienen mejores resultados que la regresión ordinaria.
* ¿Qué estrategia se podría elegir para encontrar valores de la regulzación que mejoren el desempeño del modelo en los datos de prueba?

### 4.2 Arboles de decisión

En contraste con los modelos de regresión, los árboles de decisión no suponen ninguna relación funcional entre las características y la variable objetivo, en cambio se basan en dividir al espacio en regiones numéricas (cortes) que sirven para crear predicciones.

En esencia, un árbol de decisión comienza con un nodo raíz que representa a todo el conjunto de datos. Luego, el modelo divide los datos en subconjuntos más pequeños basados en las características más importantes que influyen en la predicción de la variable de salida. 

**Figura 1:** *Diagrama del algoritmo para construir un árbol de decisión*
![title](../images/decision_tree.png)

Cada subconjunto se divide en subconjuntos más pequeños hasta que se alcanza un nivel de profundidad predeterminado o cuando se cumple un criterio de parada, como la cantidad mínima de muestras requeridas en un nodo o la falta de mejora en la precisión de la predicción.

Una vez que se ha construido el árbol de decisión, se utiliza para predecir la variable de salida de nuevas observaciones. El modelo sigue el árbol de decisión, comenzando en el nodo raíz y siguiendo las ramas hasta llegar a un nodo hoja que proporciona la predicción numérica.

Por ejemplo si queremos predecir el salario de un jugador de la NBA basado en su altura y el numero de rebotes por juego, podríamos dividir los datos regiones por altura (mayores a 1.80 m, entre 1.80 m y 1.60, asi como menores de 1.60 m). Y a su vez dividir por la cantidad de rebotes que los jugadores generan en rangos.


**Figura 2:** *Ejemplo de árbol de decisión para regresión*
![title](../images/regresion_tree_example.png)

La documentación de SKlearn es a su vez una buena referencia de como se construyen los árboles de decisión https://inria.github.io/scikit-learn-mooc/python_scripts/trees_regression.html

Cabe destacar que los árboles de decisión pueden ser muy complejos, pues pueden integrar gran número nodos, subnodos y hojas dentro de ellos para dividir el espacio y ajustarse a la explicación de un fenómeno, especialmente si es no lineal. En este sentido, para conocer si la división de un corte en los arboles se introducen medidas de calidad de la división realiza, que esencialmente nos dan un medición de si estamos ganando información para explicar mejor un fenómeno a realizar una nueva división del espacio; si reducimos la calidad este corte será aceptable y viceversa si aumenta la impureza será mejor no considerarlo.

En este sentido, los principales hiper-parámetros de un árbol de decisión son:

* **Profundidad máxima del árbol:** se refiere la cantidad de divisiones que se permiten en el árbol. Cuanto más profundo sea el árbol, más complejo y detallado será el modelo. Sin embargo, si el árbol es demasiado profundo, es más probable que se ajuste demasiado al conjunto de datos de entrenamiento y no se generalice bien a nuevos datos.
* **Criterio de división**: esto determina cómo se mide la calidad de una división en el árbol. El criterio más común es la reducción de una métrica como en RSME, pero son aceptables otros tipo de errores como el MAE.
* **Mínimo de muestas por hoja:** esto controla la cantidad mínima de muestras que se requieren para que un nodo sea considerado una hoja en el árbol. Si el número de muestras es menor que este valor, el nodo no se dividirá más y se convertirá en una hoja. Esto puede ayudar a prevenir el ajuste excesivo al conjunto de datos de entrenamiento.
* **Mínimo de muestras para dividir:** parámetro que estable la cantidad mínima de muestras que se requieren para que un nodo sea candidato para la división. Si el número de muestras es menor que este valor, el nodo no se dividirá más. Esto puede ayudar a prevenir la creación de ramas con pocos datos y que no sean representativas del conjunto de datos completo.

Ahora bien esta misma capacidad de los árboles de crecer para adaptarse a la disposición de las variables en el espacio para predecir un fenómeno, también puede llevar a que sobre ajusten, por lo que existen técnicas para construir árboles de decisión, revisar su estructura para decidir acortarlos en una región o establecer valores mínimos de cuantos puntos deben mantenerse en cada hoja, dichos valores se consideran hiper-parámetros de los arboles de decisión.

Para mayor referencia consultar la documentación de Sklear: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html

Se encuentran disponibles en Skleanr mediante la clase `DecisionTreeRegressor`.

## 4.3 Bosques Aleatorios (Random Forest)

Como vimos, los árboles de decisión se pueden adaptar para explicar fenómenos no lineales, sin embargo pueden tener sobreajuste. Para lidiar con ellos, se inventó una técnica estadística llamada *Bagging*, donde se toma un conjunto de datos de entrenamiento y se construyen muchos árboles de decisión independientes utilizando diferentes subconjuntos de características y observaciones aleatorias del conjunto de datos. 

Una vez construidos estos modelos, para predecir el valor de un nuevo punto de datos se construye la predicción individual de cada árbol para este punto y se toma el promedio de sus predicciones. El resultado final es una predicción de regresión más precisa que la que se obtendría de cualquier árbol de decisión individual, ya que el modelo utiliza la combinación de múltiples árboles para reducir el sesgo y la varianza de las predicciones.

**Figura 3:** *Esquema de un bosque aleatorio.*
![title](../images/random_forest.png)

Los principales parámetros que se utilizan para ajustar el modelo son:

* **Número de árboles:** se refiere al número de árboles que se construirán en el modelo. A medida que se aumenta este número, el modelo puede volverse más preciso pero también puede aumentar el tiempo de entrenamiento.
* **Profundidad máxima del árbol:** se refiere a la profundidad máxima que se permitirá que tenga cada árbol en el modelo. A medida que se aumenta la profundidad máxima, el modelo puede volverse más preciso pero también puede aumentar el riesgo de sobreajuste.
* **Cantidad de características:** se refiere al número de características que se tomarán en consideración al dividir cada nodo del árbol. A medida que se aumenta este número, el modelo puede volverse más preciso pero también puede aumentar el tiempo de entrenamiento.
* **Cantidad de individuos por hoja:** relativo al número mínimo de muestras que se requieren para formar una hoja en cada árbol del modelo. A medida que se aumenta este número, el modelo puede volverse más conservador y menos propenso a sobreajustarse, pero también puede disminuir la precisión.

En Sklearn, se encuentra disponibles a través de la clase `RandomForestClassifier`. Ver la documentación correspondiente: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

A continuación se mostrará un ejemplo para predecir la demanda de BikerPro usando Arboles de Decisión y Bosques Aleatorios.


In [203]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

# Instanciamos los diferentes modelos
dt_1 = DecisionTreeRegressor(
    max_depth=3,
    min_samples_split=5,
    min_samples_leaf=5
    )

dt_2 = DecisionTreeRegressor(
    max_depth=5,
    min_samples_split=7,
    min_samples_leaf=6
    )

dt_3 = DecisionTreeRegressor(
    max_depth=20,
    min_samples_split=10,
    min_samples_leaf=25
    )


rf = RandomForestRegressor(
    n_estimators=100,
    criterion='squared_error',
    max_depth=10, min_samples_split=5,
    min_samples_leaf=2,
    )

models_trees_regresion = {
    'decisioin_tree_1': dt_1,
    'decisioin_tree_2': dt_2,
    'decisioin_tree_3': dt_3,
    'random_forest': rf
    }

In [204]:
for model_name in models_trees_regresion.keys():

    print("Modelo:", model_name)
    model = models_trees_regresion[model_name]

    # Ajusta el modelo con los datos de prueba
    model.fit(X_train[weather_cols],y_train)

    y_train_pred = model.predict(X_train[weather_cols])
    y_test_pred = model.predict(X_test[weather_cols])

    # error en conjunto de entrenamiento y prueba
    error_train = mean_squared_error(y_train, y_train_pred)
    error_test = mean_squared_error(y_test, y_test_pred)

    # errores
    print("Error RSME en train:", round(error_train,2) )
    print("Error RSME en test:", round(error_test,2) )

    print("----------------------------------------------")


Modelo: decisioin_tree_1
Error RSME en train: 229305.92
Error RSME en test: 328860.31
----------------------------------------------
Modelo: decisioin_tree_2
Error RSME en train: 187141.84
Error RSME en test: 313706.05
----------------------------------------------
Modelo: decisioin_tree_3
Error RSME en train: 135837.74
Error RSME en test: 307116.74
----------------------------------------------
Modelo: random_forest
Error RSME en train: 83448.74
Error RSME en test: 299236.68
----------------------------------------------


**Preguntas**
* ¿Cuál de los tres modelos tuvo mejor desempeño? ¿Porqué sudecio eso?

## 5. Validación Cruzada y selección del mejor modelo

Como hemos visto, al presentarse modelos con mayor complejidad, estos suelen acompañarse por **hiper-parámetros**, es decir, parámetros que no se ajustan automáticamente durante el entrenamiento y necesitan ser especificados de antemano. Ello representa un dilema, pues no tenemos forma de saber como se desempeñara un modelo ante datos que nunca ha visto.

Para ello, existe una técnica denominada **validación cruzada**, que, en términos sencillos, consiste en dividir los datos en varios conjuntos de entrenamiento y prueba, y entrenar y evaluar el modelo varias veces utilizando diferentes combinaciones de estos conjuntos. Esto nos da una mejor estimación de la capacidad del modelo para generalizar a datos nuevos y no vistos.

Esta técnica se utiliza en la calibración de hiperparámetros para evitar el sobreajuste y seleccionar los mejores valores de los hiperparámetros que maximicen el rendimiento en los datos no vistos durante la evaluación. Es decir, al tener varios conjuntos de entrenamiento y prueba, se puede aproximar el comportamiento de una modelo a través de diferentes combinaciones de valores de sus hiper-parámetros y con ello selección los que tienen un mayor desempeño, que típicamente se obtiene promediando el valor de la función de pérdida en todos los subconjuntos de entrenamiento y prueba generados y tomando el que tiene mejor desempeño.

En la práctica la construcción de los conjuntos de entrenamiento para validación cruzada esencialmente se basa en tomar construir subconjuntos disjuntos del conjunto de entrenamiento. Básicamente estos subconjuntos se toman como una nueva versión de datos para entrenar y probar modelos, cuidando que no sucede fuga de datos.

En particular en problemas como el de BikePro donde existe una componente tempora, una forma de construir los conjuntos de validación cruza se realiza recorriendo ventanas de entrenamiento y un de datos de prueba en futuro inmediato del primer conjunto, de manera que esta ventanas se van recorriendo hacia adelante y con ello se evita que el modelo se entrene con datos que no pertenecen al futuro en donde va a evular su desempeño.

**Figura 1:** *Esquema de los datos de validación cruzada con componentes temporales.*
![title](../images/temporal_cross_validation.png)


En Python, existe una clase que nos permite construir esta clase de conjuntos de validación llamada `TimeSeriesSplit`. El resto de estrategias de validación cruzada se puede consultar en la documentación de SKlearn https://scikit-learn.org/stable/modules/cross_validation.html

In [205]:
from sklearn.model_selection import TimeSeriesSplit

# Datos para probar los indices de TimeSeriesSplit
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4], [1, 2], [3, 4], [3, 4], [1, 2], [3, 4], [3, 4]])
y = np.array([1, 2, 3, 4, 5, 6,7,8,9,10])

# Instanciamos la validación cruazada para TimeSeriesSplit
# Notas: se generan 4 conjuntos con datos de prueba de 
# tamaño 2 hacie el futuro
tscv = TimeSeriesSplit(n_splits=4, test_size=2)

# Imprime los valores de los indices
for train_index, test_index in tscv.split(X):
   print("Entrenamiento:", train_index, "Prueba:", test_index)

Entrenamiento: [0 1] Prueba: [2 3]
Entrenamiento: [0 1 2 3] Prueba: [4 5]
Entrenamiento: [0 1 2 3 4 5] Prueba: [6 7]
Entrenamiento: [0 1 2 3 4 5 6 7] Prueba: [8 9]


## 5.1 Calibración de hiper parámetros con validación cruzada

Permite realizar la calibración de hiper parámetros de una manera sencilla empleando la clase `GridSearchCV` (véase https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html), ya que unicamente tenemos que pasarle:
* el modelo o el pipeline que queremos ajustar,
* un diccionario indicando los hiper parámetros del modelo (con un formato específico, a discutir en breve)
* el esquema de validación cruzada (en nuestro caso es la instancia de `TimeSeriesSplit`)
* el nombre métrica necesaria para evaluar el ajuste (ver la sección **Regression** de https://scikit-learn.org/stable/modules/model_evaluation.html).

Y con ello se encargará de probar todas las combinaciones de parámetros que le proporcionemos para regresarnos posteriormente el modelo con el mejor desempeño en los subconjuntos de validación cruzada. Dicho modelo, se debe evaluar aun en el conjunto de prueba ya con los mejores hiper-parámetros elegidos en el proceso de validación cruzada.




### 5.1.1 **Ejemplo de validación cruzada**

En este ejemplo ajustaremos un modelo de regresión de Lasso, como sabemos toma un hiper parámetros de nombre alpha, con la métrica RMSE. El conjunto de validación será de dos meses, equivalente a 1440 observaciones hacia el futuro.



In [206]:
# importamos la libreria GridSearchCV
from sklearn.model_selection import GridSearchCV

Definimos el modelo del que queremos ajustar los hiper parámetros en el conjunto de entrenamiento usando validación cruzada, en este caso es una regresión de Lasso:

In [207]:
# modelo de regresion de laso
reg_lasso = Lasso(
    alpha=0.0001 ,
    max_iter= 3000,
    random_state=0
    )

Ahora definimos un diccionario con los valores del hiper parámetro a probar:

In [208]:
parameters = {
    'alpha': [1e-15,1e-13,1e-10,1e-8,1e-5,1e-4,1e-3,1e-2,1e-1,1,5,10,20,30,40,45,50,55,60,100,0.0014]
    }

Escogemos el nombre de la métrica, en este caso es RSME, denominada como `neg_root_mean_squared_error` en la documentación https://scikit-learn.org/stable/modules/model_evaluation.html.

Por otro lado, creamos el conjunto de indices para la validación cruzada, indicando el número de conjunto de entrenamiento, así como el tamaño del conjunto de entrenamiento (1440):

In [209]:
n_splits = 5
tscv = TimeSeriesSplit(n_splits, test_size=1440)

Comunicamos esta información a GridSearchCV

In [210]:
model_lasso = GridSearchCV(
    reg_lasso,
    parameters,
    n_jobs=-1,
    scoring='neg_mean_squared_error',
    cv=tscv)

Ahora entrenamos el modelo y lo evaluamos:

In [211]:
weather_cols = [
    'temperature',
    'humidity',
    'wind_speed',
    'visibility',
    'dew_point_temperature',
    'solar_radiation',
    'rainfall',
    'snowfall',
 ]

model_lasso.fit(X_train[weather_cols], y_train)

/Users/cesar/opt/anaconda3/envs/aument_conda_1/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.704e+06, tolerance: 4.668e+02
  model = cd_fast.enet_coordinate_descent(
/Users/cesar/opt/anaconda3/envs/aument_conda_1/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.704e+06, tolerance: 4.668e+02
  model = cd_fast.enet_coordinate_descent(
/Users/cesar/opt/anaconda3/envs/aument_conda_1/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, chec

GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=1440),
             estimator=Lasso(alpha=0.0001, max_iter=3000, random_state=0),
             n_jobs=-1,
             param_grid={'alpha': [1e-15, 1e-13, 1e-10, 1e-08, 1e-05, 0.0001,
                                   0.001, 0.01, 0.1, 1, 5, 10, 20, 30, 40, 45,
                                   50, 55, 60, 100, 0.0014]},
             scoring='neg_mean_squared_error')

Evaluando el modelo:

In [212]:
y_train_pred = model_lasso.predict(X_train[weather_cols])
y_test_pred = model_lasso.predict(X_test[weather_cols])

# error en conjunto de entrenamiento y prueba
error_train = mean_squared_error(y_train, y_train_pred)
error_test = mean_squared_error(y_test, y_test_pred)

# errores
print("Error RSME en train:", round(error_train,2) )
print("Error RSME en test:", round(error_test,2) )

Error RSME en train: 242664.74
Error RSME en test: 297240.06


También podemos acceder a lo resultados de la validación cruzada:

In [213]:
print(" Resultados" )
print("\n Mejor modelo en calibración :\n", model_lasso.best_estimator_)
print("\n Mejor métrica de evaluación:\n", model_lasso.best_score_)
print("\n Parámetro con mejor desempeño:\n", model_lasso.best_params_)


 Resultados

 Mejor modelo en calibración :
 Lasso(alpha=1, max_iter=3000, random_state=0)

 Mejor métrica de evaluación:
 -274961.81268149265

 Parámetro con mejor desempeño:
 {'alpha': 1}


**Preguntas**
* ¿Cuál fue el hiper parámetro que dió el mejor modelo?

### 5.1.2 Ejemplo de validación cruzada usando Pipeline

La validación cruzada también de puede aplicar usando un procesamiento de datos con Pipeline, el flujo casi igual al anterior, con la excepción de que la espeficicación del diccinario que indar el hiper parámetro se modificará un poco: se deberá añadir el nombre del modelo con un sufijo de doble guión bajo.

Ahora veremos un ejemplo de este flujo, empleando el ejemplo del notebook anterior:

**Importamos librerias**

In [214]:
from sklearn.feature_selection import SelectKBest, r_regression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

**Definimos las columnas del modelo**

In [215]:
# Define listas de columnas que van a emplearse en el modelado
weather_cols = [
    'temperature',
    'humidity',
    'wind_speed',
    'visibility',
    'dew_point_temperature',
    'solar_radiation',
    'rainfall',
    'snowfall',
 ]

seasons_cols = ['seasons']

time_cols = ['hour']

# Lista que tiene todas los grupos de columnas
non_target_cols = weather_cols + seasons_cols + time_cols

**Creamos el pipeline de pre-procesamiento**

Notas:
    * En `pipe_standard_ohe` se ha definido un modelo Lasso con nombre `model`
    * En `model__alpha` se ha espeficido el valor del parámetro usando `model__alpha`, es decir agregamos al inicio el nombre que le dimos al modelo en el pipeline más doble guion bajo.

In [216]:
# Pipeline para escalar con estandar z-score
numerical_pipe = Pipeline([
    ('standar_scaler', StandardScaler()),
    # ----------- Aqui seleccionamos las 4 mejores variables -------- #
    ('select_k_best',SelectKBest(r_regression, k=4) ),
])

# Pipeline para aplicar one hot encoding
categorical_pipe = Pipeline([
    ('one_hot', OneHotEncoder(handle_unknown='ignore'))
])

# Combina ambos procesos en columnas espeficadas en listas
pre_processor = ColumnTransformer([
    ('numerical', numerical_pipe, weather_cols),
    ('categorical', categorical_pipe, seasons_cols),
], remainder='passthrough')

# comunica al pipeline la lista en el orden que se deben aplicar
# estos pasos

pipe_standard_ohe = Pipeline([
    ('transform', pre_processor),
    # Define modelo lasso
    ('model', Lasso(alpha=0.0001 , max_iter= 3000, random_state=0))
])

# Diccionario con el nombre del modelo
param_grid = {
    'model__alpha': [1e-15,1e-13,1e-10,1e-8,1e-5,1e-4,1e-3,1e-2,1e-1,1,5,10,20,30,40,45,50,55,60,100,0.0014]
}

lasso_reg_2 = GridSearchCV(
    pipe_standard_ohe,
    param_grid,
    n_jobs=-1,
    scoring='neg_mean_squared_error',
    cv=tscv
    )

In [217]:
# Realiza la transformacion de los datos y el ajuste del modelo
lasso_reg_2.fit(X_train[non_target_cols], y_train)

/Users/cesar/opt/anaconda3/envs/aument_conda_1/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/cesar/opt/anaconda3/envs/aument_conda_1/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/cesar/opt/anaconda3/envs/aument_conda_1/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/cesar/opt/anaconda3/envs/aument_conda_1/lib/python3.10/site-packages/sklearn/utils/validation.py

GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=1440),
             estimator=Pipeline(steps=[('transform',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('numerical',
                                                                         Pipeline(steps=[('standar_scaler',
                                                                                          StandardScaler()),
                                                                                         ('select_k_best',
                                                                                          SelectKBest(k=4,
                                                                                                      score_func=<function r_regression at 0x7fb9c8e65ea0>))]),
                                                                         ['temperature',
                                                                          'hu...
                                                                          'solar_radiation',
                                                                          'rainfall',
                                                                          'snowfall']),
                                                                        ('categorical',
                                                                         Pipeline(steps=[('one_hot',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['seasons'])])),
                                       ('model',
                                        Lasso(alpha=0.0001, max_iter=3000,
                                              random_state=0))]),
             n_jobs=-1,
             param_grid={'model__alpha': [1e-15, 1e-13, 1e-10, 1e-08, 1e-05,
                                          0.0001, 0.001, 0.01, 0.1, 1, 5, 10,
                                          20, 30, 40, 45, 50, 55, 60, 100,
                                          0.0014]},
             scoring='neg_mean_squared_error')

**Evaluamos el modelo**

In [218]:
y_train_pred = lasso_reg_2.predict(X_train[non_target_cols])
y_test_pred = lasso_reg_2.predict(X_test[non_target_cols])

# error en conjunto de entrenamiento y prueba
error_train = mean_squared_error(y_train, y_train_pred)
error_test = mean_squared_error(y_test, y_test_pred)

# errores
print("Error RSME en train:", round(error_train,2) )
print("Error RSME en test:", round(error_test,2) )

Error RSME en train: 216705.71
Error RSME en test: 250756.72


In [219]:
print(" Resultados" )
print("\n Mejor modelo en calibración :\n", lasso_reg_2.best_estimator_)
print("\n Mejor métrica de evaluación:\n", lasso_reg_2.best_score_)
print("\n Parámetro con mejor desempeño:\n", lasso_reg_2.best_params_)

 Resultados

 Mejor modelo en calibración :
 Pipeline(steps=[('transform',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numerical',
                                                  Pipeline(steps=[('standar_scaler',
                                                                   StandardScaler()),
                                                                  ('select_k_best',
                                                                   SelectKBest(k=4,
                                                                               score_func=<function r_regression at 0x7fb9c8e65ea0>))]),
                                                  ['temperature', 'humidity',
                                                   'wind_speed', 'visibility',
                                                   'dew_point_temperature',
                                                   'solar_radiation',
                       

### 5.1.1 **Ejemplo de validación cruzada con Pipeline y más de un modelo**

Ahora mostraremos con la estructura anterior se puede usar para ajustar los hiper-parámetros de varios modelos a la vez. El código es casi igual, solo hay que especificar los modelos y sus hiper parámetros en forma particular.

**Especificamos el pipeline de procesamiento**

Aquí indicaremos un modelo cualquiera

In [265]:
# Pipeline para escalar con estandar z-score
numerical_pipe = Pipeline([
    ('standar_scaler', StandardScaler()),
    # ----------- Aqui seleccionamos las 4 mejores variables -------- #
    ('select_k_best',SelectKBest(r_regression, k=4) ),
])

# Pipeline para aplicar one hot encoding
categorical_pipe = Pipeline([
    ('one_hot', OneHotEncoder(handle_unknown='ignore'))
])

# Combina ambos procesos en columnas espeficadas en listas
pre_processor = ColumnTransformer([
    ('numerical', numerical_pipe, weather_cols),
    ('categorical', categorical_pipe, seasons_cols),
], remainder='passthrough')

# comunica al pipeline la lista en el orden que se deben aplicar
# estos pasos

pipe_standard_ohe = Pipeline([
    ('pre_processro', pre_processor),
    # Define modelo lasso
    ('model', RandomForestRegressor())
])

**Especificaciones de modelos y parámetros**

Definimos los modelos a probar:

In [266]:
model1 =  Lasso()
model2 = RandomForestRegressor()

*Modelo 1: Regresión de Lasso*

Especificamos los hiper-parámetros del modelo y la instancia del modelo lasso

In [267]:
params1 = {}
params1['model__alpha'] = [1e-15,1e-13,1e-10,1e-8,1e-5,1e-4,1e-3,1e-2,1e-1,1,5,10,20,30,40,45,50,55,60,100,0.0014]
params1['model__max_iter'] = [1500, 3000]
params1['model'] = [model1] # <- modelo dentro de una lista

**Modelo 2: Bosque Aleatorio**

Nuevamente especificamos los hiper-parámetros del modelo y la instancia del modelo. Los hiper parámetros se pueden consultar aqui: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html#sklearn.ensemble.RandomForestRegressor

In [268]:
params2 = {}
params2['model__n_estimators'] = [3, 4, 5, 10],
params2['model__max_features'] = ["auto", "sqrt", "log2"]
params2['model__min_samples_split'] = [3, 4, 5, 10]
params2['model__bootstrap'] = [True, False]
params2['model'] = [model2] # <- modelo dentro de una lista

**Ahora generamos una lista de los diccionarions de parámetros y modelos**

In [269]:
params_multi = [params1, params2]

In [270]:
model_csv_multi = GridSearchCV(
    pipe_standard_ohe,
    params_multi,
    n_jobs=-1,
    scoring='neg_mean_squared_error',
    cv=tscv
    )

In [271]:
# Realiza la transformacion de los datos y el ajuste del modelo
model_csv_multi.fit(X_train[non_target_cols], y_train)


/Users/cesar/opt/anaconda3/envs/aument_conda_1/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/cesar/opt/anaconda3/envs/aument_conda_1/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.459e+06, tolerance: 4.668e+02
  model = cd_fast.enet_coordinate_descent(
/Users/cesar/opt/anaconda3/envs/aument_conda_1/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/cesar/opt/

GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=1440),
             estimator=Pipeline(steps=[('pre_processro',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('numerical',
                                                                         Pipeline(steps=[('standar_scaler',
                                                                                          StandardScaler()),
                                                                                         ('select_k_best',
                                                                                          SelectKBest(k=4,
                                                                                                      score_func=<function r_regression at 0x7fb9c8e65ea0>))]),
                                                                         ['temperature'...
                          'model__alpha': [1e-15, 1e-13, 1e-10, 1e-08, 1e-05,
                                           0.0001, 0.001, 0.01, 0.1, 1, 5, 10,
                                           20, 30, 40, 45, 50, 55, 60, 100,
                                           0.0014],
                          'model__max_iter': [1500, 3000]},
                         {'model': [RandomForestRegressor()],
                          'model__bootstrap': [True, False],
                          'model__max_features': ['auto', 'sqrt', 'log2'],
                          'model__min_samples_split': [3, 4, 5, 10],
                          'model__n_estimators': ([3, 4, 5, 10],)}],
             scoring='neg_mean_squared_error')

**Evaluamos el modelo**

In [272]:
y_train_pred = model_csv_multi.predict(X_train[non_target_cols])
y_test_pred = model_csv_multi.predict(X_test[non_target_cols])

# error en conjunto de entrenamiento y prueba
error_train = mean_squared_error(y_train, y_train_pred)
error_test = mean_squared_error(y_test, y_test_pred)

# errores
print("Error RSME en train:", round(error_train,2) )
print("Error RSME en test:", round(error_test,2) )

Error RSME en train: 216705.71
Error RSME en test: 250756.72


In [273]:
print(" Resultados" )
print("\n Mejor modelo en calibración :\n", model_csv_multi.best_estimator_)
print("\n Mejor métrica de evaluación:\n", model_csv_multi.best_score_)
print("\n Parámetro con mejor desempeño:\n", model_csv_multi.best_params_)

 Resultados

 Mejor modelo en calibración :
 Pipeline(steps=[('pre_processro',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numerical',
                                                  Pipeline(steps=[('standar_scaler',
                                                                   StandardScaler()),
                                                                  ('select_k_best',
                                                                   SelectKBest(k=4,
                                                                               score_func=<function r_regression at 0x7fb9c8e65ea0>))]),
                                                  ['temperature', 'humidity',
                                                   'wind_speed', 'visibility',
                                                   'dew_point_temperature',
                                                   'solar_radiation',
                   

**Pregunta**
    * ¿cual modelo tuvo desempeño?

### 6. Entregables

A. Modificando el código anterior, crea un script que entrene los modelos KNN, regresión de Ridge y Bosque Aleatoriorealizando validación cruzada de sus hiper-parámetros, sobre los conjuntos de entrenamiento y pruebas descritos en este documento. Se espera que realizando ingenieria de características, selección de variables y calibración de hiper-parámetros alcance un valor de RMSE cercano a las 200 unidades.

**Hint:** Puedes usar las variables y pre-procesamiento que quieras, pero estas son sugerencias de lo que tu pipeline podría integrar:

    * Variables:
      * Variables numéricas del clima (todas a excepción de `dew_point_temperature`)
      * Variable categóricas con one-hot encoding:
        * seasons
        * hour (para transforma a categoria puede usar el comando `data['hour].astype('category')`)
        * dia de la semana (revisa la sección de feature engineering y conviertela en categoria)
      * Variables categoricas binarias:
        * Holiday
        * functional day
        * indicador de fin de semana

Este script deberá deberá denominarse `model_prediction_bikerpro.py` y deberá salvar el modelo resultante en formato picke denominado `model_prediction_bikerpro.pk`

B. Este script deberá generar dos gráficas que comparen los valores reales de la demanda de rente de bicicletas de BikerPro, tanto en los conjuntos de entrenamiento como de prueba
ambas en formato .png con lo nombres `comparative_actual_model_train_set.png` y `comparative_actual_model_test_set.png`